In [30]:
import sqlite3
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
def load_db(db_file):
  with sqlite3.connect(db_file) as conn:
    tables = list(pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)['name'])
    db = {table : pd.read_sql_query(f"SELECT * from {table}", conn) for table in tables}
  return db

In [32]:
db_rewiews = load_db("reviews.db")

In [33]:
db_rewiews['reviews'].sample(2)

,Date,GoodNum,Rating
45388,2020-08-01,167,1
39974,2020-03-01,45,4


In [34]:
db_rewiews['reviews'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49024 entries, 0 to 49023
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     49024 non-null  object
 1   GoodNum  49024 non-null  int64 
 2   Rating   49024 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [35]:
db_rewiews['reviews'].describe()

,GoodNum,Rating
count,49024.000000,49024.000000
mean,64.302056,2.671936
std,54.011630,1.344051
min,0.000000,1.000000
25%,25.000000,1.000000
50%,48.000000,3.000000
75%,88.000000,4.000000
max,189.000000,5.000000


In [36]:
reviews = db_rewiews['reviews'].copy()
categs = db_rewiews['categs'].copy()
goods = db_rewiews['goods'].copy()

In [37]:
goods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   GoodNum        100 non-null    int64 
 1   GoodDesc       100 non-null    object
 2   ProductCatNum  100 non-null    int64 
dtypes: int64(2), object(1)
memory usage: 2.5+ KB


#Анализ отзывов

1. Средний рейтинг по **продуктам**.

In [38]:
products_avg_rating = (
    reviews
    .merge(goods)
    .groupby('GoodDesc', as_index=False)
    .agg(AvgRating=('Rating', 'mean'))
)
products_avg_rating

,GoodDesc,AvgRating
0,Аргосульфан крем туба 2% 40г,3.184932
1,"Африн Экстро спрей наз. 0,05% 15мл",1.714286
2,"Африн спрей наз. 0,05% 15мл",4.339394
3,Африн увлаж. спрей назал. фл. 0.05% 15 мл,2.133333
4,Банеоцин мазь туба 20г,1.733333
...,...,...
95,"Хлоргексидин тюб.-кап.0,05% 10 мл №5",4.386018
96,"Хлоргексидин фл. 0,05% 100мл (пластик)",2.122486
97,"Хлоргексидина биглюконат фл. 0,05% 100мл (плас...",3.502616
98,"Хлоргексидина биглюконат фл. 0,05% 100мл (плас...",3.045181


2. **Продукты** с наибольшим количеством отрицательных отзывов (рейтинг 1–2).

In [39]:
products_with_max_quantity_low_rating = (
  reviews
  .query('Rating in (1, 2)')
  .merge(goods)
  .groupby('GoodDesc', as_index=False)
  .agg(CountLowMark=('Rating', 'count'))
  .sort_values('CountLowMark', ascending=False)
)
products_with_max_quantity_low_rating.head(5)

,GoodDesc,CountLowMark
24,Корвалол фл.25мл,1540
22,Контейнер с крышкой 120мл стер. (д/мочи) инд. уп.,1146
84,"Снуп спрей 0,1% 15мл",1118
64,Омепразол (Омез)-Тева капс. 20мг №28,891
72,Пенталгин таб. №24,790


3. **Продукты** с наименьшим рейтингом.

In [40]:
products_with_low_rating = products_avg_rating.sort_values('AvgRating').head(5)

1. Средний рейтинг по **категориям**

In [41]:
categories_avg_rating = (
    reviews
    .merge(goods)
    .merge(categs)
    .groupby('ProductCatDesc', as_index=False)
    .agg(AvgRating=('Rating', 'mean'))
)
categories_avg_rating

,ProductCatDesc,AvgRating
0,Анальгетики Бренды,2.417292
1,Антисептические средства Хлоргексидин,3.124918
2,Гематогены/мюсли,2.877469
3,Контейнер для анализов,2.598909
4,Насморк лечение КСИЛОМЕТАЗОЛИН спрей,2.442000
5,ОМЕПРАЗОЛ,2.169521
6,Пластыри бактерицидные,2.831796
7,Противомикробные наружные средства,2.842919
8,"Седативные, настойки/капли",2.742927
9,Сосудосуживающие для взрослых спрей,2.711376


2. **Категории** с наибольшим количеством отрицательных отзывов (рейтинг 1–2).

In [42]:
categories_with_max_quantity_low_rating = (
  reviews
  .query('Rating in (1, 2)')
  .merge(goods)
  .merge(categs)
  .groupby('ProductCatDesc', as_index=False)
  .agg(CountLowMark=('Rating', 'count'))
  .sort_values('CountLowMark', ascending=False)
)
categories_with_max_quantity_low_rating.head(5)

,ProductCatDesc,CountLowMark
4,Насморк лечение КСИЛОМЕТАЗОЛИН спрей,4387
0,Анальгетики Бренды,3746
8,"Седативные, настойки/капли",3435
2,Гематогены/мюсли,1972
6,Пластыри бактерицидные,1912


3. **Категории** с наименьшим рейтингом.

In [43]:
categories_avg_rating.sort_values('AvgRating').head(5)

,ProductCatDesc,AvgRating
5,ОМЕПРАЗОЛ,2.169521
0,Анальгетики Бренды,2.417292
4,Насморк лечение КСИЛОМЕТАЗОЛИН спрей,2.442000
3,Контейнер для анализов,2.598909
9,Сосудосуживающие для взрослых спрей,2.711376


In [44]:
with pd.ExcelWriter('task2.xlsx', engine='openpyxl') as writer:
  products_avg_rating.rename(
    columns={'GoodDesc':"Название продукта", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Продажи по продуктам', index=False)
  products_with_max_quantity_low_rating.head(5).rename(
    columns={'GoodDesc':"Название продукта", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Продажи по продуктам', index=False)
  products_with_low_rating.head(5).rename(
    columns={'GoodDesc':"Название продукта", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Продажи по продуктам', index=False)

  categories_avg_rating.rename(
    columns={'ProductCatDesc':"Название категории", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Средний рейтинг по категориям', index=False)
  categories_with_max_quantity_low_rating.head(5).rename(
    columns={'ProductCatDesc':"Название категории", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Средний рейтинг по категориям', index=False)
  categories_avg_rating.head(5).rename(
    columns={'ProductCatDesc':"Название категории", 'AvgRating':'Средний рейтинг'})\
    .to_excel(writer, sheet_name='Средний рейтинг по категориям', index=False)

#Дополнительные задания

Продукты, у которых больше остальных выросли и упали средние оценки.
Были выбраны следующие метрики:
  * Размах
  * Отношение масимального и минимального рейтингов
  * Разница между средним рейтнгом в начале и конце периода
  * Отношение между средним рейтнгом в конце и начале периода

In [45]:
with sqlite3.connect('reviews.db') as conn:
  query = """
  WITH avg_rating_window as (
    SELECT
      GoodDesc,
      Date,
      AVG(Rating) OVER (PARTITION BY GoodNum ORDER BY Date) as Avg,
      FIRST_VALUE(AVG(Rating)) OVER (PARTITION BY GoodDesc ORDER BY Date) as first_avg,
      LAST_VALUE(AVG(Rating)) OVER (PARTITION BY GoodDesc ORDER BY Date) as last_avg
    FROM reviews AS r
      JOIN goods AS g USING(GoodNum)
    GROUP BY GoodDesc, Date
  )

  SELECT
    GoodDesc,
    MAX(Avg) - MIN(Avg) as diff,
    MAX(Avg)/MIN(Avg) as rel,
    last_avg - first_avg as diff_first_last,
    last_avg/first_avg as rev_diff_first_last
  FROM avg_rating_window
  GROUP BY GoodDesc
  ORDER BY diff_first_last DESC, GoodDesc
  """
  df = pd.read_sql(query, conn)
  print(df)

                                             GoodDesc      diff       rel  \
0                     Омепразол (Омез) капс. 20мг №28  0.625000  1.234375   
1                                   Валемидин фл.25мл  0.840580  1.360248   
2           Лейкопластырь бактерицидный 3,8см х 3,8см  1.191667  1.744792   
3                            Риностоп спрей 0,1% 15мл  0.666667  1.242424   
4   Контейнер с крышкой 120мл стер. (д/мочи) инд. уп.  0.904762  1.633333   
..                                                ...       ...       ...   
95  Хлоргексидина биглюконат фл. 0,05% 100мл (плас...  1.428571  1.476190   
96  Лейкопластырь бактерицидный набор №20 (беж. ос...  1.416667  1.435897   
97        Гематоген "Русский"* вак. уп. 40г (детский)  1.608696  1.672727   
98                   Солпадеин Фаст таб. №12 (раств.)  2.083333  1.714286   
99                         Нурофен таб. п/о 200мг №20  2.083333  1.714286   

    diff_first_last  rev_diff_first_last  
0          0.750000             

Категории, у которых больше остальных выросли и упали средние оценки.
Были выбраны следующие метрики:
  * Размах
  * Отношение масимального и минимального рейтингов
  * Разница между средним рейтнгом в начале и конце периода
  * Отношение между средним рейтнгом в конце и начале периода

In [46]:
with sqlite3.connect('reviews.db') as conn:
  query = """
  WITH avg_rating_window as (
    SELECT
      GoodNum,
      ProductCatNum,
      ProductCatDesc,
      Date,
      AVG(Rating) OVER (PARTITION BY ProductCatDesc ORDER BY Date) as Avg,
      FIRST_VALUE(AVG(Rating)) OVER (PARTITION BY ProductCatDesc ORDER BY Date) as first_avg,
      LAST_VALUE(AVG(Rating)) OVER (PARTITION BY ProductCatDesc ORDER BY Date) as last_avg
    FROM reviews AS r
      JOIN goods AS g USING(GoodNum)
        JOIN categs AS c USING(ProductCatNum)
    GROUP BY ProductCatDesc, Date
  )

  SELECT
    ProductCatDesc,
    MAX(Avg) - MIN(Avg) as diff,
    MAX(Avg)/MIN(Avg) as rel,
    last_avg - first_avg as diff_first_last,
    last_avg/first_avg as rev_diff_first_last
  FROM avg_rating_window
  GROUP BY ProductCatDesc
  ORDER BY diff_first_last DESC
  """
  df = pd.read_sql(query, conn)
  print(df)

                          ProductCatDesc      diff       rel  diff_first_last  \
0                 Контейнер для анализов  0.904762  1.633333         0.521286   
1                              ОМЕПРАЗОЛ  1.541667  2.027778         0.242765   
2    Сосудосуживающие для взрослых спрей  1.565217  1.455696         0.059677   
3             Седативные, настойки/капли  0.250000  1.250000         0.000000   
4     Противомикробные наружные средства  1.333333  2.333333         0.000000   
5                 Пластыри бактерицидные  1.333333  1.666667         0.000000   
6   Насморк лечение КСИЛОМЕТАЗОЛИН спрей  0.045455  1.045455         0.000000   
7                       Гематогены/мюсли  1.833333  2.833333         0.000000   
8  Антисептические средства Хлоргексидин  1.333333  1.500000        -0.132982   
9                     Анальгетики Бренды  0.666667  1.500000        -0.395483   

   rev_diff_first_last  
0             1.350373  
1             1.137510  
2             1.022182  
3       

#Выводы
Наибольшее количество негативных отзывов получили:

* Продукты: "Корвалол фл.25мл" (1540 отзывов), "Контейнер с крышкой 120мл стер." (1146 отзывов), "Снуп спрей 0,1% 15мл" (1118 отзывов).

* Категории: "Насморк лечение КСИЛОМЕТАЗОЛИН спрей" (4387 отзывов), "Анальгетики Бренды" (3746 отзывов), "Седативные, настойки/капли" (3435 отзывов).

Динамика рейтингов:

* Продукты: Наибольший рост среднего рейтинга наблюдается у "Омепразол (Омез) капс. 20мг №28" (разница: 0.75), а наибольшее падение — у "Нурофен таб. п/о 200мг №20" (разница: -3.27).

* Категории: "Контейнер для анализов" показал наибольший рост (разница: 0.52), тогда как "Анальгетики Бренды" — наибольшее падение (разница: -0.40).